In [ ]:
from keras.applications.resnet_v2 import ResNet50V2
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import keras
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten

import numpy as np
import tensorflow as tf
import os, pathlib
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# TO-DO: add argparse when converting to script
num_classes = 4 
batch_size = 8
epochs = 100
lr = 0.00002 # 0.00002
checkpoint = ''
outputPath = './output/'
runID = 'pneu-lr' + str(lr)
runPath = outputPath + runID
pathlib.Path(runPath).mkdir(parents=True, exist_ok=True)
print('Output: ' + runPath)

Output: ./output/pneu-lr2e-05


In [3]:
# load data
x_train = np.load('data/x_train_3.npy')
x_test = np.load('data/x_test_3.npy')
y_train = np.load('data/y_train_3.npy')
y_test = np.load('data/y_test_3.npy')

In [4]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
def get_callbacks(runPath):
    callbacks = []
    lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=0.000001, min_delta=1e-2)
    callbacks.append(lr_schedule) # reduce learning rate when stuck

    # Callback: save checkpoints '/cp-{epoch:02d}-{val_loss:.2f}.ckpt'
    checkpoint_path = runPath + '/cp-{epoch:02d}-{val_loss:.2f}.hdf5'
    callbacks.append(keras.callbacks.ModelCheckpoint(checkpoint_path,
        verbose=1, save_best_only=False, save_weights_only=True, mode='min', period=1))

    class SaveAsCKPT(keras.callbacks.Callback):
        def __init__(self):
            self.saver = tf.train.Saver()
            self.sess = keras.backend.get_session()

        def on_epoch_end(self, epoch, logs=None):
            checkpoint_path = runPath + '/cp-{:02d}.ckpt'.format(epoch)
            save_path = self.saver.save(self.sess, checkpoint_path)
    callbacks.append(SaveAsCKPT())

    return callbacks


In [6]:
# TO-DO: add checkpoint
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
__________

In [7]:
opt = Adam(learning_rate=lr, amsgrad=True)
callbacks = get_callbacks(runPath)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy']) # TO-DO: add additional metrics for COVID-19
print('Ready for training!')
datagen = ImageDataGenerator(rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                             # randomly shift images horizontally (fraction of total width)
                             width_shift_range=0.1,
                             # randomly shift images vertically (fraction of total height)
                             height_shift_range=0.1,
                             brightness_range=[0.8,1.2], # 1.0 means no change, >1 increase in brightness
                             # set mode for filling points outside the input boundaries
                             fill_mode='nearest',
                             cval=0.,  # value used for fill_mode = "constant"
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

class_weight = {0: 2.,
                1: 2.,
                2: 1.,
                3: 50.}

# Fit the model on the batches generated by datagen.flow().
'''model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    callbacks=callbacks,
                    epochs=epochs,
                    class_weight=class_weight,
                    validation_data=(x_test, y_test))'''
model.fit(x_train, y_train, batch_size=batch_size, callbacks=callbacks, epochs=epochs, class_weight=class_weight, validation_data=(x_test, y_test))

Ready for training!

Train on 3955 samples, validate on 403 samples
Epoch 1/100
3955/3955 [==============================] - 102s 26ms/step - loss: 1.4946 - accuracy: 0.7226 - val_loss: 0.5212 - val_accuracy: 0.7717

Epoch 00001: saving model to ./output/pneu-lr2e-05/cp-01-0.52.hdf5
Epoch 2/100
3955/3955 [==============================] - 96s 24ms/step - loss: 0.5988 - accuracy: 0.8278 - val_loss: 0.6421 - val_accuracy: 0.7543

Epoch 00002: saving model to ./output/pneu-lr2e-05/cp-02-0.64.hdf5
Epoch 3/100
3955/3955 [==============================] - 98s 25ms/step - loss: 0.2658 - accuracy: 0.9209 - val_loss: 0.4623 - val_accuracy: 0.8586

Epoch 00003: saving model to ./output/pneu-lr2e-05/cp-03-0.46.hdf5
Epoch 4/100
3955/3955 [==============================] - 98s 25ms/step - loss: 0.1181 - accuracy: 0.9709 - val_loss: 0.5719 - val_accuracy: 0.8065

Epoch 00004: saving model to ./output/pneu-lr2e-05/cp-04-0.57.hdf5
Epoch 5/100
3955/3955 [==============================] - 97s 25ms/step 

3955/3955 [==============================] - 97s 24ms/step - loss: 0.0028 - accuracy: 0.9997 - val_loss: 0.6206 - val_accuracy: 0.8784

Epoch 00036: saving model to ./output/pneu-lr2e-05/cp-36-0.62.hdf5
Epoch 37/100
3955/3955 [==============================] - 97s 25ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 0.6506 - val_accuracy: 0.8586

Epoch 00037: saving model to ./output/pneu-lr2e-05/cp-37-0.65.hdf5
Epoch 38/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6421 - val_accuracy: 0.8561

Epoch 00038: saving model to ./output/pneu-lr2e-05/cp-38-0.64.hdf5
Epoch 39/100
3955/3955 [==============================] - 97s 25ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.6317 - val_accuracy: 0.8710

Epoch 00039: saving model to ./output/pneu-lr2e-05/cp-39-0.63.hdf5
Epoch 40/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: 0.6119 - val_accuracy: 0.8660



Epoch 74/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 0.6121 - val_accuracy: 0.8685

Epoch 00074: saving model to ./output/pneu-lr2e-05/cp-74-0.61.hdf5
Epoch 75/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6235 - val_accuracy: 0.8610

Epoch 00075: saving model to ./output/pneu-lr2e-05/cp-75-0.62.hdf5
Epoch 76/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0027 - accuracy: 0.9995 - val_loss: 0.6524 - val_accuracy: 0.8660

Epoch 00076: saving model to ./output/pneu-lr2e-05/cp-76-0.65.hdf5
Epoch 77/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0026 - accuracy: 0.9992 - val_loss: 0.6071 - val_accuracy: 0.8710

Epoch 00077: saving model to ./output/pneu-lr2e-05/cp-77-0.61.hdf5
Epoch 78/100
3955/3955 [==============================] - 97s 24ms/step - loss: 0.0021 - accuracy: 0.9997 - val_loss: 0.6269 - val_accur

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [8]:
from sklearn.metrics import confusion_matrix

'''base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights(runPath + '/cp-48-0.59.hdf5')
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])'''

y_pred = model.predict(x_test)
#matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [9]:
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
cm_norm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [10]:
print(matrix)
class_acc = np.array(cm_norm.diagonal())
print(class_acc)
print('Normal: {0:.3f}, Bacterial: {1:.3f}, Viral: {2:.3f}, COVID-19: {3:.3f}'.format(class_acc[0],
                                                                                      class_acc[1],
                                                                                      class_acc[2],
                                                                                      class_acc[3]))

[[237   5   4]
 [ 44 105   0]
 [  0   0   8]]
[0.96341463 0.70469799 1.        ]


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
import csv

sess = tf.Session()
saver = tf.train.import_meta_graph(runPath + '/cp-49.ckpt.meta')
saver.restore(sess, runPath + '/cp-49.ckpt')

graph = tf.get_default_graph()

with open('graph.csv', mode='w') as graph_file:
    writer = csv.writer(graph_file)
    for tensor in tf.contrib.graph_editor.get_tensors(graph):
        writer.writerow([str(tensor.name), tensor.shape])

writer = tf.summary.FileWriter(runPath, sess.graph)
writer.close()